In [1]:
import os
import pandas as pd
import numpy as np
import random
import pickle

import warnings
warnings.filterwarnings('ignore')

%config Completer.use_jedi = False

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
df = pd.read_csv('../data/traindata_demo.csv')
df.dropna(subset=['sapo'], inplace=True)
df

,uid,articleID,sapo
0,1002681074,4243622,Galaxy S21 Ultra được trang bị pin 5.000 mAh n...
1,1002681074,4243459,"Thấy nòng súng súng thò ra khỏi cửa, nữ cảnh s..."
2,1002681074,4238172,"Sau khi có chức vô địch Australia Mở rộng, Nov..."
3,1002681074,4237883,Novak Djokovic đạt tỷ lệ thắng 81% trước các đ...
4,1002681074,4231206,Một người không thể đi hơn một xe máy ra đường...
...,...,...,...
2726,1003347668,4154347,Trên chương trình El Larguero của đài Cadena S...
2727,1003347668,4154384,Tiền đạo Robert Lewandowski nói anh xứng đáng ...
2728,1003347668,4155165,Hầu hết những chỉ số tấn công của Lionel Messi...
2729,1003347668,4155219,Indonesia


## bỏ qua các bước preprocessing, chỉ cần tới bước load data and model là có dữ liệu clean sẵn

## Preprocessing

In [ ]:
# lấy user mà clicked articleID => article khác của user khác
def get_negative_sample(articleID, df, npratio=4):
    uid = df[df.articleID==articleID].uid.values[0]
    tmp_df = df[df.uid != uid].sample(4)
    return [articleID] + list(tmp_df.articleID)

In [ ]:
def preprocess_clicked(df, npratio=4):
    uids = df.uid.unique()
    user_count = len(uids)
    userid_dict = {}
    for uid in uids:
        if uid not in userid_dict:
            userid_dict[uid] = len(userid_dict) # map uid_raw -> 0 1 2

    all_train_id = []
    all_train_pn = []
    all_label = []
    
    all_test_id = []
    all_test_pn = []
    all_test_label = []
    all_test_index = []
    
    all_user_pos = []
    all_test_user_pos = []

    for uid in uids:
        tmp_df = df[df.uid==uid] # df of uid
        clicked_news = tmp_df.articleID.values
        clicked_news = set(clicked_news)  # get all unique article which user clicked
        
        for idx in range(len(tmp_df)-1):
            line = tmp_df.iloc[idx]
            all_train_pn.append(get_negative_sample(line.articleID, df))
            all_label.append([1,0,0,0,0])
            all_train_id.append(userid_dict[uid])

            remain_clicked = list(clicked_news - set([line.articleID]))
            remain_clicked = random.sample(remain_clicked, min(50, len(remain_clicked)))
            remain_clicked += [0] * (50-len(remain_clicked)) # <50 cho = 0
            all_user_pos.append(remain_clicked)
        
        # get the last line for testing
        sess_index = []
        sess_index.append(len(all_test_pn))

        line = tmp_df.iloc[-1]
        all_test_pn += get_negative_sample(line.articleID, df)
        sess_index.append(len(all_test_pn))
        all_test_index.append(sess_index)
        all_test_label += [1,0,0,0,0]
        all_test_id += [userid_dict[uid]] * (npratio+1)
        allpos = random.sample(clicked_news, min(50, len(clicked_news)))
        allpos += [0] * (50-len(allpos))
        for i in range(5):
            all_test_user_pos.append(allpos)
    
    all_train_pn = np.array(all_train_pn,dtype='int32')
    all_label = np.array(all_label,dtype='int32')
    all_train_id = np.array(all_train_id,dtype='uint64')
    all_test_pn = np.array(all_test_pn,dtype='int32')
    all_test_label = np.array(all_test_label,dtype='int32')
    all_test_id = np.array(all_test_id,dtype='uint64')
    all_user_pos = np.array(all_user_pos,dtype='int32')
    all_test_user_pos = np.array(all_test_user_pos, dtype='int32')

    return (userid_dict, user_count, all_train_pn, all_label, all_train_id, all_test_pn, all_test_label, all_test_id, all_user_pos, all_test_user_pos, all_test_index)

In [ ]:
%%time
userid_dict,user_count, all_train_pn, all_label, all_train_id, all_test_pn, all_test_label, all_test_id, all_user_pos, all_test_user_pos, all_test_index = preprocess_clicked(df)

In [ ]:
dataloader = (user_count, all_train_pn, all_label, all_train_id, all_test_pn, all_test_label, all_test_id, all_user_pos, all_test_user_pos, all_test_index)

file = open('./model/thuytt_ver2/dataloader.pkl', 'wb')
pickle.dump(dataloader, file)
file.close()

In [ ]:
file = open('./model/thuytt_ver2/dataloader.pkl', 'rb')
user_count, all_train_pn, all_label, all_train_id, all_test_pn, all_test_label, all_test_id, all_user_pos, all_test_user_pos, all_test_index = pickle.load(file)
file.close()

## Tokenizer and make word_dict of articles

In [ ]:
from vncorenlp import VnCoreNLP
VnCoreNLP_jar_file = '../../vncorenlp/VnCoreNLP-1.1.1.jar'
rdrsegmenter = VnCoreNLP(VnCoreNLP_jar_file, annotators='wseg')
embedding_dim=768

In [ ]:
def preprocess_news(df):
    sapos = df.sapo.values
    articleIds = df.articleID.values

    news = {} 

    for i in range(len(articleIds)):
        if articleIds[i] not in news:
            tokenized_words = rdrsegmenter.tokenize(sapos[i])[0]
            news[articleIds[i]] = tokenized_words

    
    word_dict_raw = {'PADDING': [0,999999]}
    for articleId in news:
        for word in news[articleId]:
            if word in word_dict_raw:
                word_dict_raw[word][1] += 1 # increase freq
            else:
                word_dict_raw[word] = [len(word_dict_raw), 1] # format: [index, freq]
                
    word_dict = {}
    for i in word_dict_raw:
        if word_dict_raw[i][1] >= 2:
            word_dict[i] = [len(word_dict), word_dict_raw[i][1]]
    print('len word_dict (freq>=2 vs raw):', len(word_dict), len(word_dict_raw)) # chỉ để so sánh (loại bỏ freq =1)
    
    print('leng news (tokenizer):',len(news))

    news_words = [ [0]*30 ] # 
    news_index = {0:0}
    
    for articleId in news: # quét các article
        word_id = []
        news_index[articleId] = len(news_index)
        for word in news[articleId]: # quét các tokens
            if word in word_dict:
                word_id.append(word_dict[word][0])
        word_id = word_id[:30] # lấy word_id của article (embedd)
        news_words.append(word_id + [0]*(30-len(word_id))) # max 30 tokens, <30 cho =0
    
    news_words = np.array(news_words, dtype='int32')

    return word_dict, news_words, news_index, news

In [ ]:
%%time
word_dict, news_words, news_index, news = preprocess_news(df)

In [ ]:
file = open('./model/thuytt_ver2/phobert_news_preprocess.pkl', 'wb')
pickle.dump((word_dict, news_words, news_index, news), file)
file.close()

In [ ]:
file = open('./model/thuytt_ver2/phobert_news_preprocess.pkl', 'rb')
word_dict, news_words, news_index, news = pickle.load(file)
file.close()

## Phobert + embedding_matrix

In [ ]:
import argparse

# from transformers import RobertaConfig, RobertaModel

from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary

In [ ]:
vocab = Dictionary()
vocab.add_from_file('../PhoBERT_base_transformers/dict.txt')

In [ ]:
# load bert model
from fairseq.models.roberta import RobertaModel
import os
phobert = RobertaModel.from_pretrained('../PhoBERT_base_fairseq', checkpoint_file='model.pt')
phobert.eval()  # disable dropout (or leave in train mode to finetune)

# Incorporate the BPE encoder into PhoBERT-base 
from fairseq.data.encoders.fastbpe import fastBPE  
from fairseq import options  
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes', 
    default="../PhoBERT_base_transformers/bpe.codes",
    required=False,
    type=str,
    help='path to fastBPE BPE'
)
args, unknown = parser.parse_known_args()
phobert.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT

In [ ]:
def get_embedding(word_dict):
    embedding_dict = {}

    for word in word_dict:
        input_ids = vocab.encode_line(word, append_eos=False, add_if_not_exist=False).long()
        embedding_tensor = phobert.extract_features(input_ids)
        embedding_dict[word] = embedding_tensor.data.cpu().numpy()[0][0]
        
    embedding_matrix = [0]*len(word_dict)
    cand = []

    for i in embedding_dict:
        embedding_matrix[word_dict[i][0]] = np.array(embedding_dict[i], dtype='float32')
        cand.append(embedding_matrix[word_dict[i][0]])
    
    cand = np.array(cand, dtype='float32')
    mu = np.mean(cand, axis=0)
    Sigma = np.cov(cand.T)
    norm = np.random.multivariate_normal(mu, Sigma, 1)

    for i in range(len(embedding_matrix)):
        if type(embedding_matrix[i]) == int: # unknown words
            embedding_matrix[i] = np.reshape(norm, embedding_dim)
    
    embedding_matrix[0] = np.zeros(embedding_dim, dtype='float32')
    embedding_matrix = np.array(embedding_matrix, dtype='float32')

    print(embedding_matrix.shape)
    return embedding_matrix

In [ ]:
%%time
embedding_mat = get_embedding(word_dict)

In [ ]:
file = open('./model/thuytt_ver2/phobert_embed_mat.pkl', 'wb')
pickle.dump(embedding_mat, file)
file.close()

In [ ]:
file = open('./model/thuytt_ver2/phobert_embed_mat.pkl', 'rb')
embedding_mat = pickle.load(file)
file.close()
embedding_dim=768

## Load data and model

In [4]:
# load thuytt_ver2
working_dir = '/home/thuytt/test_bert/NCKH/model/thuytt_ver2/'
os.chdir(working_dir)

file = open('dataloader.pkl', 'rb')
num_users, all_train_pn, all_label, all_train_id, all_test_pn, all_test_label, all_test_id, all_user_pos, all_test_user_pos, all_test_index = pickle.load(file)
file.close()

file = open('phobert_news_preprocess.pkl', 'rb')
word_dict, news_words, news_index, news = pickle.load(file)
file.close()

file = open('phobert_embed_mat.pkl', 'rb')
embedding_mat = pickle.load(file)
file.close()

In [5]:
print(embedding_mat.shape, embedding_mat.shape[1])

(3111, 768) 768


# Model

## Metric

In [5]:
def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)

## generate batch

In [6]:
# hàm để convert articles word sang dạng index dựa vào từ điển word_dict
def _articles_to_index(arr_np):
    arr_index_articles = []
    for arr_articles in arr_np:
        index_article = []
        for article in arr_articles:
            index_article.append(news_index[article])
        arr_index_articles.append(index_article)
    arr_index_articles = np.array(arr_index_articles, dtype='int32')
    return arr_index_articles

In [7]:
def generate_batch_data_train(all_train_pn,all_label,all_train_id,all_user_pos,batch_size):
    inputid = np.arange(len(all_label))
    np.random.shuffle(inputid)
    y=all_label
    batches = [inputid[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            if(i.size ==0):
                continue
            index_all_train_pn = _articles_to_index(all_train_pn[i])
            candidate = news_words[index_all_train_pn]
            candidate_split=[candidate[:,k,:] for k in range(candidate.shape[1])]

            #
            index_all_user_pos = _articles_to_index(all_user_pos[i])
            browsed_news=news_words[index_all_user_pos]
            browsed_news_split=[browsed_news[:,k,:] for k in range(browsed_news.shape[1])]
            userid=np.expand_dims(all_train_id[i],axis=1)
            label=all_label[i]
            yield (candidate_split + browsed_news_split + [userid], label)

In [9]:
def generate_batch_data_test(all_test_pn,all_test_label,all_test_id,all_test_user_pos,batch_size):
    inputid = np.arange(len(all_test_label))
    y=all_test_label
    batches = [inputid[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            if(i.size ==0):
                continue
            index_all_test_pn = [news_index[x] for x in all_test_pn[i]]
            candidate = news_words[index_all_test_pn]

            browsed_news=news_words[_articles_to_index(all_test_user_pos[i])]
            browsed_news_split=[browsed_news[:,k,:] for k in range(browsed_news.shape[1])]
            userid=np.expand_dims(all_test_id[i],axis=1)
            label=all_test_label[i]

            yield ([candidate] + browsed_news_split + [userid], label)

## Keras model

In [8]:
import keras
from keras.layers import *
from keras.models import Model
from keras import backend as K
from keras.optimizers import *
from sklearn.metrics import roc_auc_score
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

Using TensorFlow backend.


In [ ]:
MAX_SENT_LENGTH=30 # số lượng tokens cho mỗi article
MAX_SENTS=50 # maximum clicked article for user embedding
npratio=4
batch_size=32
n_epoch=100

In [9]:
# seed theo code của a hiếu, e cũng chưa biết để làm gì nên cứ kệ để đó
def seed_everything(SEED):
    np.random.seed(SEED)
    tf.compat.v1.set_random_seed(SEED)
    
seed_everything(42)

In [ ]:
print(len(word_dict))

In [ ]:
results=[]

# user embedding
user_id = Input(shape=(1,), dtype='uint64')

user_embedding_layer= Embedding(user_count, MAX_SENTS, trainable=True) #ouput leng == max clicked_news
user_embedding= user_embedding_layer(user_id)
user_embedding_word= Dense(200,activation='relu')(user_embedding)
user_embedding_word= Flatten()(user_embedding_word)
user_embedding_news= Dense(200,activation='relu')(user_embedding)
user_embedding_news= Flatten()(user_embedding_news)


# news embedding architecture
news_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedding_layer = Embedding(len(word_dict) , embedding_dim, weights=[embedding_mat],trainable=True)
embedded_sequences = embedding_layer(news_input)
embedded_sequences =Dropout(0.2)(embedded_sequences)

cnnouput = Conv1D(padding='same', activation='relu', strides=1, filters=embedding_dim, kernel_size=3)(embedded_sequences)
cnnouput=Dropout(0.2)(cnnouput) # layer mình cần lấy để làm vector article như đã bàn

attention_a = Dot((2, 1))([cnnouput, Dense(embedding_dim,activation='tanh')(user_embedding_word)])
attention_weight = Activation('softmax')(attention_a)
news_rep=keras.layers.Dot((1, 1))([cnnouput, attention_weight])
newsEncoder = Model([news_input,user_id], news_rep)


# clicked news embedding
all_news_input = [keras.Input((MAX_SENT_LENGTH,), dtype='int32') for _ in range(MAX_SENTS)]
browsed_news_rep = [newsEncoder([news,user_id]) for news in all_news_input]
browsed_news_rep = concatenate([Lambda(lambda x: K.expand_dims(x,axis=1))(news) for news in browsed_news_rep],axis=1)


# User Embedding
attention_news = keras.layers.Dot((2, 1))([browsed_news_rep, Dense(embedding_dim,activation='tanh')(user_embedding_news)])
attention_weight_news = Activation('softmax')(attention_news)
user_rep = keras.layers.Dot((1, 1))([browsed_news_rep, attention_weight_news])


# candidate news embedding
candidates = [keras.Input((MAX_SENT_LENGTH,), dtype='int32') for _ in range(1+npratio)]
candidate_vecs = [ newsEncoder([candidate,user_id]) for candidate in candidates]

# Click Predictor???
logits = [keras.layers.dot([user_rep, candidate_vec], axes=-1) for candidate_vec in candidate_vecs]
logits = keras.layers.Activation(keras.activations.softmax)(keras.layers.concatenate(logits))


model = Model(candidates+all_news_input+[user_id], logits)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.00025), metrics=['acc'])


candidate_one = keras.Input((MAX_SENT_LENGTH,))
candidate_one_vec = newsEncoder([candidate_one,user_id])
score = keras.layers.Activation(keras.activations.sigmoid)(keras.layers.dot([user_rep, candidate_one_vec], axes=-1))
model_test = keras.Model([candidate_one]+all_news_input+[user_id], score)

In [15]:
class NPA:
    def __init__(self, num_users, max_sents, max_sent_length, 
                 embedding_mat, npratio = 4, dense_dim = 200, lr = 0.00025):
        self.__num_users = num_users
        self.__max_sents = max_sents
        self.__max_sent_length = max_sent_length
        self.__embedding_mat = embedding_mat
        
        self.__embedding_dim = embedding_mat.shape[1]
        self.__dense_dim = dense_dim
        self.__npratio = npratio
        
        self.__lr = lr
        
        # User embeddings
        self.user_input = Input(shape=(1,), dtype='uint64')
        user_embedding_layer = Embedding(self.__num_users, self.__max_sents, trainable=True) #ouput leng == max clicked_news
        user_embedding = user_embedding_layer(self.user_input)
        
        user_embedding_word = Dense(self.__dense_dim, activation='relu')(user_embedding)
        user_embedding_word = Flatten()(user_embedding_word)
        
        user_embedding_news = Dense(self.__dense_dim, activation='relu')(user_embedding)
        user_embedding_news = Flatten()(user_embedding_news)
        
        # News embedding 
        self.news_input = Input(shape=(self.__max_sent_length,), dtype='int32')
        embedding_layer = Embedding(self.__embedding_mat.shape[0], self.__embedding_dim, 
                                    weights=[self.__embedding_mat],trainable=True) # ?? Co train hay giu nguyen?
        embedded_sequences = embedding_layer(self.news_input)
        embedded_sequences =Dropout(0.2)(embedded_sequences)
        
        cnn_layer = Conv1D(padding='same', activation='relu', strides=1, 
                          filters = self.__embedding_dim, kernel_size=3)(embedded_sequences)
        cnnoutput = Dropout(0.2)(cnn_layer)
        self.new_embedding_extractor = Model([self.news_input, self.user_input], cnnoutput)
        
        attention_a = Dot((2, 1))([cnnoutput, Dense(self.__embedding_dim, activation='tanh')(user_embedding_word)])
        attention_weight = Activation('softmax')(attention_a)
        news_rep = keras.layers.Dot((1, 1))([cnnoutput, attention_weight])
        
        self.news_encoder = Model([self.news_input, self.user_input], news_rep)
        
        # clicked news embedding
        clicked_news_input = [keras.Input((self.__max_sent_length,), dtype='int32') for _ in range(MAX_SENTS)]
        clicked_news_rep = [self.news_encoder([news, self.user_input]) for news in clicked_news_input]
        clicked_news_rep = concatenate([Lambda(lambda x: K.expand_dims(x, axis=1))(news) for news in clicked_news_rep],axis=1)

        # User Embedding
        attention_news = keras.layers.Dot((2, 1))(
            [clicked_news_rep, Dense(self.__embedding_dim, activation='tanh')(user_embedding_news)])
        attention_weight_news = Activation('softmax')(attention_news)
        user_rep = keras.layers.Dot((1, 1))([clicked_news_rep, attention_weight_news])

        # candidate news embedding
        candidate_news_input = [keras.Input((self.__max_sent_length,), dtype='int32') for _ in range(1 + self.__npratio)]
        candidate_vecs = [self.news_encoder([candidate, self.user_input]) for candidate in candidate_news_input]

        # Click probability
        logits = [keras.layers.dot([user_rep, candidate_vec], axes=-1) for candidate_vec in candidate_vecs]
        logits = keras.layers.Activation(keras.activations.softmax)(keras.layers.concatenate(logits))

        self.trainer = Model(candidate_news_input + clicked_news_input + [self.user_input], logits)
        self.trainer.compile(loss='categorical_crossentropy', optimizer = Adam(self.__lr), metrics=['acc'])
        
        candidate_one_vec = self.news_encoder([self.news_input, self.user_input])
        candidate_score = Activation(keras.activations.sigmoid)(keras.layers.dot([user_rep, candidate_one_vec], axes=-1))
        self.evaluator = Model([self.news_input] + clicked_news_input + [self.user_input], candidate_score)
    
    def train(self, data_generator, verbose=True):
        self.trainer.fit_generator(data_generator, epochs=1, steps_per_epoch=25, verbose=verbose)
        
    def evaluate(self, data_generator, steps, verbose=True):
        self.evaluator.predict_generator(data_generator, steps=steps, verbose=verbose)
        
    def extract_news_embedding(self, user_id, article):
        self.news_embedding_extractor.predict([article, user_id])
        

## Train

In [12]:
# metrhistoryt of [auc, mrr, ndcg@5, ndcg@10]
best_metric = [0., 0., 0., 0.]

In [ ]:
# test
batch_size = 32
for ep in range(2):
    traingen = generate_batch_data_train(all_train_pn, all_label, all_train_id, all_user_pos, batch_size)
    model.fit_generator(traingen, epochs=1, steps_per_epoch=25)
    testgen=generate_batch_data_test(all_test_pn, all_test_label, all_test_id, all_test_user_pos, batch_size)
    click_score = model_test.predict_generator(testgen, steps=len(all_test_id)//batch_size, verbose=1)
        
    all_auc=[]
    all_mrr=[]
    all_ndcg=[]
    all_ndcg2=[]
    for m in all_test_index:
        if np.sum(all_test_label[m[0]:m[1]])!=0 and m[1]<len(click_score):
            all_auc.append(roc_auc_score(all_test_label[m[0]:m[1]], click_score[m[0]:m[1],0]))
            all_mrr.append(mrr_score(all_test_label[m[0]:m[1]], click_score[m[0]:m[1],0]))
            all_ndcg.append(ndcg_score(all_test_label[m[0]:m[1]], click_score[m[0]:m[1],0],k=5))
            all_ndcg2.append(ndcg_score(all_test_label[m[0]:m[1]], click_score[m[0]:m[1],0],k=10))
    results.append([np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2)])
    
    metric = [np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2)]
    if metric[0] > best_metric[0] and np.mean(metric) > np.mean(best_metric):
        best_metric = metric
        print('Best model: True')
        print(metric)
    else:
        print('Best model: False')
        print(f'AUC: {metric[0]:.4f}')

In [16]:
# test

MAX_SENT_LENGTH=30 # số lượng tokens cho mỗi article
MAX_SENTS=50 # maximum clicked article for user embedding
batch_size=32
n_epoch=100

npa = NPA(num_users, MAX_SENTS, MAX_SENT_LENGTH, embedding_mat)
npa.summary()

for ep in range(2):
    traingen = generate_batch_data_train(all_train_pn, all_label, all_train_id, all_user_pos, batch_size)
    npa.train(traingen)
    testgen=generate_batch_data_test(all_test_pn, all_test_label, all_test_id, all_test_user_pos, batch_size)
    click_score = npa.evaluate(testgen, steps=len(all_test_id)//batch_size, verbose=True)
    
    all_auc=[]
    all_mrr=[]
    all_ndcg=[]
    all_ndcg2=[]
    for m in all_test_index:
        if np.sum(all_test_label[m[0]:m[1]])!=0 and m[1]<len(click_score):
            all_auc.append(roc_auc_score(all_test_label[m[0]:m[1]], click_score[m[0]:m[1],0]))
            all_mrr.append(mrr_score(all_test_label[m[0]:m[1]], click_score[m[0]:m[1],0]))
            all_ndcg.append(ndcg_score(all_test_label[m[0]:m[1]], click_score[m[0]:m[1],0],k=5))
            all_ndcg2.append(ndcg_score(all_test_label[m[0]:m[1]], click_score[m[0]:m[1],0],k=10))
    results.append([np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2)])
    
    metric = [np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2)]
    if metric[0] > best_metric[0] and np.mean(metric) > np.mean(best_metric):
        best_metric = metric
        print('Best model: True')
        print(metric)
    else:
        print('Best model: False')
        print(f'AUC: {metric[0]:.4f}')

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_115 (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
input_117 (InputLayer)          (None, 30)           0                                            
__________________________________________________________________________________________________
input_118 (InputLayer)          (None, 30)           0                                            
__________________________________________________________________________________________________
input_119 (InputLayer)          (None, 30)           0                                            
___________________________________________________________________________________________

AttributeError: 'function' object has no attribute 'summary'

In [ ]:
# save model
model_dict = {
    'model' : model,
    'model_test' : model_test
}
file = open('model_ver1.pkl', 'wb')
pickle.dump(model_dict, file)
file.close()